## Creates the doc2vec vector embeddings for a specific configuration

In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import io
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random
import gzip

from multiprocessing import Process, Queue

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *
from thesis.utils.file import *

## Global variables used throughout the script

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [4]:
MIN_WORD_COUNT = 100
NUM_CORES = 16

In [5]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [6]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"

In [7]:
root_location = "/mnt/virtual-machines/data/"
exports_location = root_location + "exported_data/"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

## Load general data required for classification

In [8]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 18.4 s, sys: 1.24 s, total: 19.6 s
Wall time: 19.6 s


In [9]:
len(training_docs_list)

1286325

In [10]:
len(validation_docs_list)

321473

In [11]:
len(test_docs_list)

401877

# Utility functions for data loading

In [12]:
VALIDATION_MINI_BATCH_SIZE = 10000
def get_extended_docs_with_inference_data_only(doc2vec_model, file_to_write, preprocessed_files_prefix, level, model_name):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_documents_reps = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
    else:
        inference_documents_reps = {}
        info("===== Getting vectors with inference")

        # Multi-threaded inference
#         inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        inference_docs_iterator = BatchWrapper(preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)
        generator_func = inference_docs_iterator.__iter__()
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        batches_run = 1
        pool = ThreadPool(NUM_CORES)
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(batches_run * mini_batch_size))
            batches_run += 1
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break
                
        pool.close()
        pool.terminate()

        pickle.dump(inference_documents_reps,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_documents_reps

In [13]:
class ExtendedPVDocumentBatchGenerator(Process):
    def __init__(self, filename_prefix, queue, batch_size=10000, start_file=0, offset=10000):
        super(ExtendedPVDocumentBatchGenerator, self).__init__()
        self.queue = queue
        self.offset = offset
        self.filename_prefix = filename_prefix
        self.files_loaded = start_file - offset

    def run(self):
        cur_file = None
        while True:
            try:
                if cur_file is None:
                    info("Loading new file for index: {}".format(str(self.files_loaded + self.offset)))
                    cur_file = io.BufferedReader(gzip.open(self.filename_prefix + str(self.files_loaded + self.offset) + '.gz'))
#                     cur_file = open(self.filename_prefix + str(self.files_loaded + self.offset))
                    self.files_loaded += self.offset
                for line in cur_file:
                    self.queue.put(line)
                cur_file.close()
                cur_file = None
            except IOError:
                self.queue.put(False, block=True, timeout=None)
                info("All files are loaded - last file: {}".format(str(self.files_loaded + self.offset)))
                return


class BatchWrapper(object):
    def __init__(self, training_preprocessed_files_prefix, buffer_size=10000, batch_size=10000, level=1, level_type=None):
        assert batch_size <= 10000 or batch_size is None
        self.level = level
        self.level_type = level_type[0] if level_type is not None else None
        self.batch_size = batch_size
        self.q = Queue(maxsize=buffer_size)
        self.p = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, queue=self.q,
                                                  batch_size=batch_size, start_file=0, offset=10000)
        self.p.start()
        self.cur_data = []

    def is_correct_type(self, doc_id):
        parts = doc_id.split("_")
        len_parts = len(parts)
        if len_parts == self.level:
            if len_parts == 1:
                return True
            if len_parts == self.level and (parts[1][0] == self.level_type or self.level_type is None):
                return True
        return False

    def return_sentences(self, line):
        line_array = tuple(line.split(" "))
        doc_id = line_array[0]
        if not self.is_correct_type(doc_id):
            return False
        line_array = line_array[1:]
        len_line_array = len(line_array)
        # divide the document to batches according to the batch size
        sentences = []
        
        if self.batch_size is None:
            # dont use LabeledSentence for validation iterator
            sentences.append((doc_id, line_array))
        else:
            curr_batch_iter = 0
            while curr_batch_iter < len_line_array:
                sentences.append(LabeledSentence(words=line_array[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id]))
                curr_batch_iter += self.batch_size
        return tuple(sentences)

    def __iter__(self):
        while True:
            item = self.q.get(block=True)
            if item is False:
                self.p.terminate()
                raise StopIteration()
            else:
                sentences = self.return_sentences(item)
                if not sentences:
                    None
                else:
                    for sentence in sentences:
                        yield sentence


# Doc2vec and SVM Parameters

In [14]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

## Create the Doc2vec model and create/load the vocab

In [15]:
models = [
    (1, 'document')
]
level, model_name = models[0]

In [17]:
info("creating/loading vocabulary for " + str(level) + ' ' + model_name + ' in ')
doc2vec_model_save_location = os.path.join(root_location,
                                           "parameter_search_doc2vec_models_recalc_" + str(level) + '_' + model_name,
                                           "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}_model_{}'.format(DOC2VEC_SIZE,
                                                                DOC2VEC_WINDOW,
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE),
                                                                str(level) + '_' + model_name
                                                                )
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
info("FILE " + os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE, window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT,
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=DOC2VEC_TRAIN_WORDS,
                iter=DOC2VEC_EPOCHS)

GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model


2017-04-19 01:57:41,817 : INFO : creating/loading vocabulary for 1 document in 
2017-04-19 01:57:41,818 : INFO : FILE /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_1_document/full/vocab_model/model


In [18]:
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX)):
    
    info("Creating vocab model")
    training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
    doc2vec_model.build_vocab(sentences=training_docs_iterator, progress_per=REPORT_VOCAB_PROGRESS)
    doc2vec_model.save(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
else:
    info("Loading vocab model")
    doc2vec_model_vocab_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
    doc2vec_model.reset_from(doc2vec_model_vocab_model)

2017-04-19 01:57:44,566 : INFO : Creating vocab model
2017-04-19 01:57:44,621 : INFO : collecting all words and their counts
2017-04-19 01:57:44,623 : INFO : Loading new file for index: 0
2017-04-19 01:57:44,643 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-04-19 01:58:14,800 : INFO : Loading new file for index: 10000
2017-04-19 01:58:42,546 : INFO : Loading new file for index: 20000
2017-04-19 01:59:11,676 : INFO : Loading new file for index: 30000
2017-04-19 01:59:41,056 : INFO : Loading new file for index: 40000
2017-04-19 02:00:10,160 : INFO : Loading new file for index: 50000
2017-04-19 02:00:39,829 : INFO : Loading new file for index: 60000
2017-04-19 02:01:04,537 : INFO : PROGRESS: at example #100000, processed 671106633 words (3357329/s), 3023524 word types, 68086 tags
2017-04-19 02:01:09,727 : INFO : Loading new file for index: 70000
2017-04-19 02:01:40,091 : INFO : Loading new file for index: 80000
2017-04-19 02:02:09,754 : INFO : Loadin

2017-04-19 02:43:34,365 : INFO : Loading new file for index: 920000
2017-04-19 02:44:03,928 : INFO : Loading new file for index: 930000
2017-04-19 02:44:33,161 : INFO : Loading new file for index: 940000
2017-04-19 02:45:02,647 : INFO : Loading new file for index: 950000
2017-04-19 02:45:25,575 : INFO : PROGRESS: at example #1400000, processed 9398449921 words (3300525/s), 20601656 word types, 957326 tags
2017-04-19 02:45:32,341 : INFO : Loading new file for index: 960000
2017-04-19 02:46:02,183 : INFO : Loading new file for index: 970000
2017-04-19 02:46:31,791 : INFO : Loading new file for index: 980000
2017-04-19 02:47:01,510 : INFO : Loading new file for index: 990000
2017-04-19 02:47:31,974 : INFO : Loading new file for index: 1000000
2017-04-19 02:48:02,375 : INFO : Loading new file for index: 1010000
2017-04-19 02:48:33,494 : INFO : Loading new file for index: 1020000
2017-04-19 02:48:51,764 : INFO : PROGRESS: at example #1500000, processed 10068733754 words (3250848/s), 2164610

## Actual Training, validation and Metrics Loop

In [19]:
doc2vec_model.min_alpha = 0.025
DOC2VEC_ALPHA_DECREASE = 0.001

In [20]:
doc2vec_model.workers = NUM_CORES

In [21]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 1
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
        training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
        %time doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
    # only do the inference for higher epochs, as inference usually takes as much time as the actual training
    if epoch == 3 or epoch == 5:
        # Validation Embeddings
        info('Getting Validation Embeddings')
        Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-19 03:02:52,121 : INFO : ****************** Epoch 1 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_1 *******************
2017-04-19 03:02:52,223 : INFO : training model with 16 workers on 446814 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=10 window=2
2017-04-19 03:02:52,224 : INFO : expecting 1879865 sentences, matching count from corpus used for vocabulary survey
2017-04-19 03:02:52,223 : INFO : Loading new file for index: 0
2017-04-19 03:02:53,231 : INFO : PROGRESS: at 0.01% examples, 485323 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:03:13,251 : INFO : PROGRESS: at 0.17% examples, 691574 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:03:33,266 : INFO : PROGRESS: at 0.34% examples, 713148 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:03:53,266 : INFO : PROGRESS: at 0.51% examples, 721711 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:04:13,271 : INFO : PROGRESS: at 0.69% exa

2017-04-19 03:24:42,790 : INFO : Loading new file for index: 150000
2017-04-19 03:24:53,720 : INFO : PROGRESS: at 11.79% examples, 767853 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:25:13,727 : INFO : PROGRESS: at 11.97% examples, 768005 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:25:33,731 : INFO : PROGRESS: at 12.14% examples, 767928 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:25:53,732 : INFO : PROGRESS: at 12.33% examples, 768033 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:26:10,060 : INFO : Loading new file for index: 160000
2017-04-19 03:26:13,735 : INFO : PROGRESS: at 12.51% examples, 768161 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:26:33,743 : INFO : PROGRESS: at 12.69% examples, 768325 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:26:53,755 : INFO : PROGRESS: at 12.87% examples, 768501 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:27:13,760 : INFO : PROGRESS: at 13.05% examples, 768666 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:27:33,763 : INFO : PROGRES

2017-04-19 03:47:54,183 : INFO : PROGRESS: at 24.34% examples, 775612 words/s, in_qsize 0, out_qsize 4
2017-04-19 03:48:14,196 : INFO : PROGRESS: at 24.52% examples, 775721 words/s, in_qsize 1, out_qsize 1
2017-04-19 03:48:34,195 : INFO : PROGRESS: at 24.71% examples, 775892 words/s, in_qsize 0, out_qsize 3
2017-04-19 03:48:54,197 : INFO : PROGRESS: at 24.89% examples, 775968 words/s, in_qsize 0, out_qsize 1
2017-04-19 03:48:58,884 : INFO : Loading new file for index: 320000
2017-04-19 03:49:14,217 : INFO : PROGRESS: at 25.08% examples, 776113 words/s, in_qsize 0, out_qsize 1
2017-04-19 03:49:34,221 : INFO : PROGRESS: at 25.26% examples, 776215 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:49:54,225 : INFO : PROGRESS: at 25.44% examples, 776322 words/s, in_qsize 0, out_qsize 2
2017-04-19 03:50:14,232 : INFO : PROGRESS: at 25.63% examples, 776425 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:50:22,088 : INFO : Loading new file for index: 330000
2017-04-19 03:50:34,238 : INFO : PROGRES

2017-04-19 04:10:54,590 : INFO : PROGRESS: at 37.21% examples, 784448 words/s, in_qsize 0, out_qsize 2
2017-04-19 04:11:09,159 : INFO : Loading new file for index: 480000
2017-04-19 04:11:14,601 : INFO : PROGRESS: at 37.40% examples, 784550 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:11:34,611 : INFO : PROGRESS: at 37.59% examples, 784680 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:11:54,618 : INFO : PROGRESS: at 37.78% examples, 784817 words/s, in_qsize 1, out_qsize 0
2017-04-19 04:12:14,620 : INFO : PROGRESS: at 37.97% examples, 784928 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:12:30,558 : INFO : Loading new file for index: 490000
2017-04-19 04:12:34,641 : INFO : PROGRESS: at 38.16% examples, 785055 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:12:54,642 : INFO : PROGRESS: at 38.35% examples, 785197 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:13:14,652 : INFO : PROGRESS: at 38.54% examples, 785360 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:13:34,654 : INFO : PROGRES

2017-04-19 04:33:55,152 : INFO : PROGRESS: at 50.28% examples, 791433 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:34:15,171 : INFO : PROGRESS: at 50.47% examples, 791522 words/s, in_qsize 0, out_qsize 3
2017-04-19 04:34:18,656 : INFO : Loading new file for index: 650000
2017-04-19 04:34:35,161 : INFO : PROGRESS: at 50.66% examples, 791595 words/s, in_qsize 0, out_qsize 3
2017-04-19 04:34:55,177 : INFO : PROGRESS: at 50.84% examples, 791662 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:35:15,177 : INFO : PROGRESS: at 51.03% examples, 791713 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:35:35,192 : INFO : PROGRESS: at 51.22% examples, 791783 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:35:42,522 : INFO : Loading new file for index: 660000
2017-04-19 04:35:55,194 : INFO : PROGRESS: at 51.41% examples, 791864 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:36:15,201 : INFO : PROGRESS: at 51.60% examples, 791950 words/s, in_qsize 0, out_qsize 2
2017-04-19 04:36:35,230 : INFO : PROGRES

...

2017-04-20 04:45:27,466 : INFO : PROGRESS: at 82.26% examples, 893028 words/s, in_qsize 0, out_qsize 2
2017-04-20 04:45:47,343 : INFO : Loading new file for index: 1060000
2017-04-20 04:45:47,466 : INFO : PROGRESS: at 82.47% examples, 893102 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:46:07,474 : INFO : PROGRESS: at 82.69% examples, 893231 words/s, in_qsize 0, out_qsize 1
2017-04-20 04:46:27,477 : INFO : PROGRESS: at 82.92% examples, 893380 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:46:47,480 : INFO : PROGRESS: at 83.14% examples, 893491 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:46:57,589 : INFO : Loading new file for index: 1070000
2017-04-20 04:47:07,482 : INFO : PROGRESS: at 83.35% examples, 893575 words/s, in_qsize 1, out_qsize 0
2017-04-20 04:47:27,486 : INFO : PROGRESS: at 83.58% examples, 893722 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:47:47,493 : INFO : PROGRESS: at 83.81% examples, 893920 words/s, in_qsize 0, out_qsize 1
2017-04-20 04:48:05,120 : INFO : Loadi

2017-04-20 05:07:47,857 : INFO : PROGRESS: at 97.11% examples, 901343 words/s, in_qsize 0, out_qsize 0
2017-04-20 05:07:53,846 : INFO : Loading new file for index: 1250000
2017-04-20 05:08:07,865 : INFO : PROGRESS: at 97.33% examples, 901431 words/s, in_qsize 0, out_qsize 1
2017-04-20 05:08:27,870 : INFO : PROGRESS: at 97.55% examples, 901532 words/s, in_qsize 0, out_qsize 0
2017-04-20 05:08:47,861 : INFO : PROGRESS: at 97.77% examples, 901596 words/s, in_qsize 0, out_qsize 0
2017-04-20 05:09:04,222 : INFO : Loading new file for index: 1260000
2017-04-20 05:09:07,865 : INFO : PROGRESS: at 97.99% examples, 901661 words/s, in_qsize 0, out_qsize 0
2017-04-20 05:09:27,868 : INFO : PROGRESS: at 98.20% examples, 901720 words/s, in_qsize 0, out_qsize 0
2017-04-20 05:09:47,870 : INFO : PROGRESS: at 98.43% examples, 901866 words/s, in_qsize 0, out_qsize 0
2017-04-20 05:10:07,871 : INFO : PROGRESS: at 98.66% examples, 901963 words/s, in_qsize 0, out_qsize 0
2017-04-20 05:10:13,431 : INFO : Loadi

CPU times: user 20h 36min 59s, sys: 12min 45s, total: 20h 49min 45s
Wall time: 2h 38min 45s


2017-04-20 05:12:10,079 : INFO : not storing attribute syn0norm
2017-04-20 05:12:10,080 : INFO : storing np array 'syn0' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model.wv.syn0.npy
2017-04-20 05:12:10,346 : INFO : storing np array 'syn1neg' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model.syn1neg.npy
2017-04-20 05:12:10,559 : INFO : not storing attribute cum_table
2017-04-20 05:12:27,408 : INFO : saved /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model


CPU times: user 7d 22h 58min 12s, sys: 1h 49min 34s, total: 8d 47min 46s
Wall time: 1d 2h 9min 35s


## Inference Only (if needed)

In [19]:
NUM_CORES = 32

In [20]:
epoch = 1
GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)

if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
    doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
    doc2vec_model.workers = NUM_CORES
    GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

2017-04-18 18:02:01,019 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_1/model
2017-04-18 18:02:08,119 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_1/model.docvecs.* with mmap=None
2017-04-18 18:02:08,121 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_1/model.docvecs.doctag_syn0.npy with mmap=None
2017-04-18 18:02:14,679 : INFO : loading wv recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean

In [22]:
GLOBAL_VARS.MODEL_NAME

'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_1'

In [21]:
doc2vec_model

In [ ]:
Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-18 18:03:06,812 : INFO : ===== Getting vectors with inference
2017-04-18 18:03:07,082 : INFO : Loading new file for index: 0
2017-04-18 18:03:26,531 : INFO : Loading new file for index: 10000
2017-04-18 18:06:37,739 : INFO : Finished: 10000 tags
2017-04-18 18:06:57,927 : INFO : Loading new file for index: 20000
2017-04-18 18:10:58,441 : INFO : Finished: 20000 tags
2017-04-18 18:11:17,440 : INFO : Loading new file for index: 30000
2017-04-18 18:15:05,454 : INFO : Finished: 30000 tags
2017-04-18 18:15:26,058 : INFO : Loading new file for index: 40000
2017-04-18 18:19:11,750 : INFO : Finished: 40000 tags
2017-04-18 18:19:32,490 : INFO : Loading new file for index: 50000
2017-04-18 18:23:17,129 : INFO : Finished: 50000 tags
2017-04-18 18:23:36,633 : INFO : Loading new file for index: 60000
2017-04-18 18:27:11,904 : INFO : Finished: 60000 tags
2017-04-18 18:27:32,721 : INFO : Loading new file for index: 70000
2017-04-18 18:31:10,730 : INFO : Finished: 70000 tags
2017-04-18 18:31:30,

#### Testing inference

In [55]:
inference_docs_iterator = BatchWrapper(validation_preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)        
for doc_tuple in inference_doczs_iterator:
    doc_id, doc_tokens = doc_tuple
    rep = doc2vec_model.infer_vector(doc_tokens)
    print (doc_id, rep)
    break

('08521002', array([  1.13558674e+00,  -2.01971769e-01,  -9.30447519e-01,
         9.55632687e-01,   5.11517346e-01,   4.34441900e+00,
        -3.77764761e-01,  -1.11617422e+00,  -2.15896085e-01,
         9.09354746e-01,   5.74674904e-01,  -2.07049704e+00,
        -7.20400810e-01,   4.94136661e-01,  -1.74060893e+00,
        -2.17272949e+00,  -4.39270258e-01,  -1.51936769e+00,
         5.65607429e-01,  -4.58835810e-01,  -1.69598356e-01,
         1.77733886e+00,   3.66123140e-01,   1.38953611e-01,
        -1.04259264e+00,   8.84979665e-01,  -8.56729895e-02,
        -6.04329109e-01,   4.42179322e-01,   1.08561194e+00,
        -2.49654725e-01,   3.02951038e-01,  -3.80307257e-01,
         1.32433748e+00,   7.18038738e-01,   7.99864233e-01,
        -3.60305488e-01,  -3.32749695e-01,   1.86409019e-02,
        -1.16298962e+00,  -2.36521304e-01,   8.52507114e-01,
        -4.25269688e-03,  -2.73190904e+00,  -1.39228487e+00,
         3.14658254e-01,   1.19927609e+00,  -8.86219382e-01,
        -2.

In [53]:
doc2vec_model.wv.syn0

array([[ 0.24962339,  0.03808838, -0.38492572, ...,  0.81019139,
        -0.0872335 ,  0.00503489],
       [-0.24393913, -0.9072656 , -0.08245134, ..., -0.12438237,
        -0.10501056,  0.07241193],
       [ 0.06769085, -0.22004843,  0.05649997, ...,  0.15331532,
        -0.87121236, -0.71148068],
       ..., 
       [ 0.02257917,  0.18380728, -0.19475998, ...,  0.72972393,
        -0.03356596, -0.29145467],
       [-0.20255305, -0.25994578,  0.31640032, ..., -0.02623975,
         0.41660461, -0.45980361],
       [ 1.09419656, -0.97489876, -0.3509953 , ...,  0.82430571,
         0.02756385,  0.7905944 ]], dtype=float32)